<a href="https://colab.research.google.com/github/Yannan-Chen/allenVisualBehaviorOphys/blob/main/dataLoader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# @title Install packages
# Might need to run twice to get everything right
!pip install pip --upgrade --quiet
!pip install mindscope_utilities --quiet
!pip install pandas --quiet
!pip install seaborn --quiet

ERROR: Could not install packages due to an EnvironmentError: [WinError 5] Access is denied: 'C:\\Users\\Yannan\\AppData\\Local\\Temp\\pip-uninstall-9xyjygpw\\pip.exe'
Consider using the `--user` option or check the permissions.

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\Yannan\\Anaconda3\\envs\\cellCount38\\Lib\\site-packages\\~andas\\_libs\\algos.cp38-win_amd64.pyd'
Consider using the `--user` option or check the permissions.



In [14]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
from glob import glob

from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.manifold import TSNE

import mindscope_utilities
import mindscope_utilities.visual_behavior_ophys as ophys

from allensdk.brain_observatory.behavior.behavior_project_cache import VisualBehaviorOphysProjectCache

pd.set_option('display.max_columns', 500)

In [4]:
data_storage_directory = r"C:\temp"  # Note: this path must exist on your local drive
cache = VisualBehaviorOphysProjectCache.from_s3_cache(cache_dir=data_storage_directory)

ophys_session_table.csv: 100%|█████████████████████████████████████████████████████| 227k/227k [00:00<00:00, 1.04MMB/s]
behavior_session_table.csv: 100%|████████████████████████████████████████████████| 1.21M/1.21M [00:00<00:00, 8.18MMB/s]
ophys_experiment_table.csv: 100%|██████████████████████████████████████████████████| 610k/610k [00:00<00:00, 4.24MMB/s]
ophys_cells_table.csv: 100%|█████████████████████████████████████████████████████| 4.29M/4.29M [00:00<00:00, 20.1MMB/s]


In [5]:
session_table = cache.get_ophys_session_table()
experiment_table = cache.get_ophys_experiment_table()

In [6]:
session_sst = session_table[['Sst' in session_table.loc[i].driver_line[0] for i in session_table.index]]
session_slc = session_table[['Slc' in session_table.loc[i].driver_line[0] for i in session_table.index]]
session_vip = session_table[['Vip' in session_table.loc[i].driver_line[0] for i in session_table.index]]

print('SST', len(session_sst))
print('Slc', len(session_slc))
print('Vip', len(session_vip))

SST 158
Slc 361
Vip 186


In [7]:
sst_sessions = np.unique(np.array(session_sst.index))
session_sst.head()

,equipment_name,full_genotype,mouse_id,reporter_line,driver_line,sex,age_in_days,cre_line,indicator,session_number,prior_exposures_to_session_type,prior_exposures_to_image_set,prior_exposures_to_omissions,behavior_session_id,ophys_experiment_id,ophys_container_id,project_code,date_of_acquisition,session_type
ophys_session_id,,,,,,,,,,,,,,,,,,,
951410079,MESO.1,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,457841,Ai148(TIT2L-GC6f-ICL-tTA2),[Sst-IRES-Cre],F,206.0,Sst-IRES-Cre,GCaMP6f,1.0,0.0,65.0,0.0,951520319,"[951980471, 951980473, 951980475, 951980479, 9...","[1018028360, 1018028345, 1018028357, 101802834...",VisualBehaviorMultiscope,2019-09-20 09:45:29.897856,OPHYS_1_images_A
952430817,MESO.1,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,457841,Ai148(TIT2L-GC6f-ICL-tTA2),[Sst-IRES-Cre],F,209.0,Sst-IRES-Cre,GCaMP6f,2.0,0.0,66.0,1.0,952554548,"[953659743, 953659745, 953659749, 953659752]","[1018028357, 1018028345, 1018028339, 1018028354]",VisualBehaviorMultiscope,2019-09-23 08:13:07.627573,OPHYS_2_images_A_passive
954954402,MESO.1,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,457841,Ai148(TIT2L-GC6f-ICL-tTA2),[Sst-IRES-Cre],F,210.0,Sst-IRES-Cre,GCaMP6f,3.0,0.0,67.0,2.0,953982960,"[958527464, 958527471, 958527474, 958527479, 9...","[1018028360, 1018028342, 1018028357, 101802835...",VisualBehaviorMultiscope,2019-09-24 16:00:00.000000,OPHYS_3_images_A
955775716,MESO.1,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,457841,Ai148(TIT2L-GC6f-ICL-tTA2),[Sst-IRES-Cre],F,212.0,Sst-IRES-Cre,GCaMP6f,3.0,1.0,68.0,3.0,956010809,"[956941841, 956941844, 956941846]","[1018028342, 1018028339, 1018028345]",VisualBehaviorMultiscope,2019-09-26 08:44:37.199778,OPHYS_3_images_A
957020350,MESO.1,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,457841,Ai148(TIT2L-GC6f-ICL-tTA2),[Sst-IRES-Cre],F,213.0,Sst-IRES-Cre,GCaMP6f,4.0,0.0,0.0,4.0,957032492,"[957759562, 957759564, 957759566, 957759570, 9...","[1018028354, 1018028345, 1018028360, 101802834...",VisualBehaviorMultiscope,2019-09-27 08:28:05.415005,OPHYS_4_images_B


In [25]:
savingLoc = r'W:\People\Yannan\allenDataset\SST'

In [24]:

for ophys_session_id in sst_sessions:

    neural_data = []
    neural_data_cleaned = pd.DataFrame({})
    
    ophys_experiment_ids = session_table.loc[ophys_session_id]['ophys_experiment_id']
    for ophys_experiment_id in ophys_experiment_ids:
        this_experiment = cache.get_behavior_ophys_experiment(ophys_experiment_id)

        this_experiment_neural_data = ophys.build_tidy_cell_df(this_experiment)

        # add some columns with metadata for the experiment
        metadata_keys = [
            'ophys_experiment_id',
            'ophys_session_id',
            'targeted_structure',
            'imaging_depth',
            'equipment_name',
            'cre_line',
            'mouse_id',
            'sex',
        ]
        for metadata_key in metadata_keys:
            this_experiment_neural_data[metadata_key] = this_experiment.metadata[metadata_key]

#         # append the data for this experiment to a list
#         neural_data.append(this_experiment_neural_data)

#     # concatate the list of dataframes into a single dataframe
#     neural_data = pd.concat(neural_data)
    
    
        all_cell_specimen_id = np.unique(this_experiment_neural_data.cell_specimen_id)

        
        for cell_specimen_id in all_cell_specimen_id:
            cell_data = this_experiment_neural_data.loc[this_experiment_neural_data.cell_specimen_id == cell_specimen_id]
            neural_data_cleaned_single = pd.concat((pd.DataFrame({
              'timestamps': [cell_data.timestamps],
              'dff': [cell_data.dff],
              'events': [cell_data.events],
              'filtered_events': [cell_data.filtered_events],
            }), pd.DataFrame(cell_data.iloc[0,4:]).transpose()), axis = 1)
            neural_data_cleaned = pd.concat([neural_data_cleaned, neural_data_cleaned_single], axis = 0)

    neural_data_cleaned.to_csv(r'\{}.csv'.format(ophys_session_id))

behavior_ophys_experiment_850517346.nwb: 100%|█████████████████████████████████████| 262M/262M [00:13<00:00, 19.0MMB/s]
behavior_ophys_experiment_850517348.nwb: 100%|█████████████████████████████████████| 242M/242M [00:10<00:00, 22.8MMB/s]
behavior_ophys_experiment_850517350.nwb: 100%|█████████████████████████████████████| 266M/266M [00:08<00:00, 32.5MMB/s]
behavior_ophys_experiment_850517352.nwb: 100%|█████████████████████████████████████| 246M/246M [00:09<00:00, 25.4MMB/s]
behavior_ophys_experiment_850517354.nwb: 100%|█████████████████████████████████████| 251M/251M [00:07<00:00, 32.0MMB/s]
behavior_ophys_experiment_850517358.nwb: 100%|█████████████████████████████████████| 254M/254M [00:10<00:00, 24.4MMB/s]
behavior_ophys_experiment_851093285.nwb: 100%|█████████████████████████████████████| 268M/268M [00:11<00:00, 22.4MMB/s]
behavior_ophys_experiment_851093287.nwb: 100%|█████████████████████████████████████| 245M/245M [00:08<00:00, 28.6MMB/s]
behavior_ophys_experiment_851093289.nwb:

behavior_ophys_experiment_958527481.nwb: 100%|█████████████████████████████████████| 249M/249M [00:08<00:00, 30.0MMB/s]
behavior_ophys_experiment_958527485.nwb: 100%|█████████████████████████████████████| 240M/240M [00:08<00:00, 29.1MMB/s]
behavior_ophys_experiment_958527488.nwb: 100%|█████████████████████████████████████| 238M/238M [00:08<00:00, 27.2MMB/s]
behavior_ophys_experiment_955276580.nwb: 100%|█████████████████████████████████████| 280M/280M [00:08<00:00, 33.8MMB/s]
behavior_ophys_experiment_956941841.nwb: 100%|█████████████████████████████████████| 266M/266M [00:09<00:00, 28.3MMB/s]
behavior_ophys_experiment_956941844.nwb: 100%|█████████████████████████████████████| 255M/255M [00:07<00:00, 33.8MMB/s]
behavior_ophys_experiment_956941846.nwb: 100%|█████████████████████████████████████| 260M/260M [00:08<00:00, 32.0MMB/s]
behavior_ophys_experiment_956903375.nwb: 100%|█████████████████████████████████████| 288M/288M [00:14<00:00, 19.5MMB/s]
behavior_ophys_experiment_957759562.nwb:

behavior_ophys_experiment_978296104.nwb: 100%|█████████████████████████████████████| 261M/261M [00:09<00:00, 27.1MMB/s]
behavior_ophys_experiment_978296110.nwb: 100%|█████████████████████████████████████| 275M/275M [00:11<00:00, 23.6MMB/s]
behavior_ophys_experiment_978296114.nwb: 100%|█████████████████████████████████████| 258M/258M [00:29<00:00, 8.68MMB/s]
behavior_ophys_experiment_978827848.nwb: 100%|█████████████████████████████████████| 288M/288M [00:13<00:00, 21.5MMB/s]
behavior_ophys_experiment_979668410.nwb: 100%|█████████████████████████████████████| 283M/283M [00:11<00:00, 25.4MMB/s]
behavior_ophys_experiment_980673831.nwb: 100%|█████████████████████████████████████| 288M/288M [00:09<00:00, 29.6MMB/s]
behavior_ophys_experiment_981178526.nwb: 100%|█████████████████████████████████████| 253M/253M [00:17<00:00, 14.4MMB/s]
behavior_ophys_experiment_981178529.nwb: 100%|█████████████████████████████████████| 257M/257M [00:12<00:00, 20.1MMB/s]
behavior_ophys_experiment_981178532.nwb:

behavior_ophys_experiment_994082656.nwb: 100%|█████████████████████████████████████| 258M/258M [00:08<00:00, 32.2MMB/s]
behavior_ophys_experiment_994082660.nwb: 100%|█████████████████████████████████████| 255M/255M [00:07<00:00, 35.8MMB/s]
behavior_ophys_experiment_994082661.nwb: 100%|█████████████████████████████████████| 255M/255M [00:07<00:00, 33.3MMB/s]
behavior_ophys_experiment_994082663.nwb: 100%|█████████████████████████████████████| 255M/255M [00:12<00:00, 20.6MMB/s]
behavior_ophys_experiment_994082664.nwb: 100%|█████████████████████████████████████| 254M/254M [00:07<00:00, 32.8MMB/s]
behavior_ophys_experiment_994082666.nwb: 100%|█████████████████████████████████████| 247M/247M [00:09<00:00, 26.5MMB/s]
behavior_ophys_experiment_994053903.nwb: 100%|█████████████████████████████████████| 330M/330M [00:09<00:00, 36.5MMB/s]
behavior_ophys_experiment_994278581.nwb: 100%|█████████████████████████████████████| 259M/259M [00:06<00:00, 38.7MMB/s]
behavior_ophys_experiment_994278583.nwb:

behavior_ophys_experiment_1076531975.nwb: 100%|████████████████████████████████████| 257M/257M [00:08<00:00, 29.3MMB/s]
behavior_ophys_experiment_1076531978.nwb: 100%|████████████████████████████████████| 254M/254M [00:09<00:00, 27.5MMB/s]
behavior_ophys_experiment_1076531981.nwb: 100%|████████████████████████████████████| 239M/239M [00:09<00:00, 24.0MMB/s]
behavior_ophys_experiment_1076531985.nwb: 100%|████████████████████████████████████| 235M/235M [00:08<00:00, 28.6MMB/s]
behavior_ophys_experiment_1076808556.nwb: 100%|████████████████████████████████████| 246M/246M [00:11<00:00, 21.9MMB/s]
behavior_ophys_experiment_1076808559.nwb: 100%|████████████████████████████████████| 246M/246M [00:09<00:00, 25.9MMB/s]
behavior_ophys_experiment_1076808560.nwb: 100%|████████████████████████████████████| 248M/248M [00:12<00:00, 20.6MMB/s]
behavior_ophys_experiment_1076808562.nwb: 100%|████████████████████████████████████| 246M/246M [00:07<00:00, 33.4MMB/s]
behavior_ophys_experiment_1076808565.nwb

behavior_ophys_experiment_1101564155.nwb: 100%|████████████████████████████████████| 231M/231M [00:06<00:00, 33.5MMB/s]
behavior_ophys_experiment_1101564161.nwb: 100%|████████████████████████████████████| 234M/234M [00:06<00:00, 34.4MMB/s]
behavior_ophys_experiment_1101564165.nwb: 100%|████████████████████████████████████| 230M/230M [00:07<00:00, 30.8MMB/s]
behavior_ophys_experiment_1101564168.nwb: 100%|████████████████████████████████████| 234M/234M [00:07<00:00, 29.8MMB/s]
behavior_ophys_experiment_1101751314.nwb: 100%|████████████████████████████████████| 244M/244M [00:06<00:00, 37.0MMB/s]
behavior_ophys_experiment_1101751316.nwb: 100%|████████████████████████████████████| 242M/242M [00:07<00:00, 30.6MMB/s]
behavior_ophys_experiment_1101751320.nwb: 100%|████████████████████████████████████| 249M/249M [01:54<00:00, 2.18MMB/s]
behavior_ophys_experiment_1101751323.nwb: 100%|████████████████████████████████████| 249M/249M [00:11<00:00, 21.6MMB/s]
behavior_ophys_experiment_1101751326.nwb